In [1]:
import pandas as pd 

In [2]:
# Base de chemin et liste des mois
base_path = "C:/Users/Jean-YvesDG/Downloads/projet_gestion_ETL_python/VENTE_USA/Donnees_csv_base/"

months = [
    "January", "February", "March", "April",
    "May", "June", "July", "August",
    "September", "October", "November", "December"
]

# Fonction pour générer les chemins de fichiers
def generate_sales_file_paths(base_path):
    return [ base_path + "Sales_" + month + "_2019.csv"for month in months]
    #
    #f"{base_path}Sales_{month}_2019.csv"


In [6]:
# Fonction pour lire et concaténer les fichiers CSV
def extract(file_list):
    df_list = []
    for file in file_list:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except FileNotFoundError:
            print(f"Fichier non trouvé : {file}")
    return pd.concat(df_list, ignore_index=True)


In [7]:
#utilisation
file_paths = generate_sales_file_paths(base_path)
final_df = extract(file_paths)

In [6]:
final_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
...,...,...,...,...,...,...
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016"
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001"
186847,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101"
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001"


In [7]:
# Étape 1 : Crée une colonne ID_unique avec des valeurs incrémentées (de 1 à nombre total de lignes)
final_df["ID_unique"] = range(1, len(final_df) + 1)

# Étape 2 : Définit cette nouvelle colonne comme index principal du DataFrame
# Cela facilite les recherches, le tri et peut servir de clé de jointure dans un entrepôt de données
final_df = final_df.set_index("ID_unique")


In [8]:
final_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
ID_unique,,,,,,
1,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
2,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
3,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
4,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
5,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
...,...,...,...,...,...,...
186846,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016"
186847,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001"
186848,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101"


renomage des colonnes


In [9]:
final_df.columns = ['Order_ID', 'Product', 'Quantity_Ordered', 'Price_Each', 'Order_Date', 'Purchase_Address']


In [10]:
#remettre ID_unique comme colonne normale
final_df = final_df.reset_index()


Merge data 


In [11]:
final_df =pd.DataFrame(final_df)

In [12]:
#load data merge
final_df.to_csv("Donnée_fusion/data_merge.CSV",index=False)


In [13]:
final_df.head()

,ID_unique,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,1,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,2,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,3,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,4,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,5,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


DECOUPAGE DE LA TABLE data_merge en plusieur tables pour faire le modele en etoile 

In [14]:

# On extrait les colonnes 'ID_unique' et 'Order_Date' de final_df
# Puis on renomme 'ID_unique' en 'ID_Date' pour la table de dimension date
# Création des tables de dimensions depuis le final_df
Dim_date = final_df[['ID_unique', 'Order_Date']].rename(columns={'ID_unique': 'ID_Date'})
Dim_adresse = final_df[['ID_unique', 'Purchase_Address']].rename(columns={'ID_unique': 'ID_Adresse'})
Dim_commande = final_df[['ID_unique', 'Order_ID', 'Product', 'Quantity_Ordered']].rename(columns={'ID_unique': 'ID_Commande'})
Ventes = final_df[['ID_unique', 'Quantity_Ordered', 'Price_Each']].rename(columns={'ID_unique': 'ID_Vente'})


In [15]:
"""# Rénomage des tables 
Dim_date.columns=['ID_Date','Order_Date']
Dim_adresse.columns=['ID_Adresse','Purchase_Address']
Dim_commande.columns=['ID_Commande','Order_ID', 'Product', 'Quantity_Ordered']
"""


"# Rénomage des tables \nDim_date.columns=['ID_Date','Order_Date']\nDim_adresse.columns=['ID_Adresse','Purchase_Address']\nDim_commande.columns=['ID_Commande','Order_ID', 'Product', 'Quantity_Ordered']\n"

In [16]:
# Fusion avec la table Dim_date
Ventes = Ventes.merge(Dim_date[['ID_Date']], left_on='ID_Vente', right_on='ID_Date', how='left')

# Fusion avec Dim_adresse
Ventes = Ventes.merge(Dim_adresse[['ID_Adresse']], left_on='ID_Vente', right_on='ID_Adresse', how='left')

# Fusion avec Dim_commande
Ventes = Ventes.merge(Dim_commande[['ID_Commande']], left_on='ID_Vente', right_on='ID_Commande', how='left')


CREATION DE NOUVEAU TABLES  - dataframe

In [17]:
Dim_date =pd.DataFrame(Dim_date)
Dim_adresse =pd.DataFrame(Dim_adresse)
Dim_commande =pd.DataFrame(Dim_commande)
Ventes =pd.DataFrame(Ventes)

TRANSFORMATION DES 4 TABLES 

Tables dimension date

In [18]:
# Conversion  de la colonne objet date en format date heure
Dim_date['Order_Date'] = pd.to_datetime(Dim_date['Order_Date'], format="%m/%d/%y %H:%M", errors="coerce")


In [19]:
# decoupage de la colonne date en deux
Dim_date['Date'] = Dim_date['Order_Date'].dt.date
Dim_date['Heure'] = Dim_date['Order_Date'].dt.time

In [20]:
# Séparer la colonne date  en année mois jours
Dim_date[['Année', 'Mois', 'Jour']] = Dim_date['Date'].astype(str).str.split('-', expand=True)


table dimension adresse

In [21]:
# Séparer la colonne en deux 
# Étape 1 : Séparer sur les virgules
Dim_adresse[['Adresse', 'Ville', 'État_CodePostal']] = Dim_adresse['Purchase_Address'].astype(str).str.split(',', expand=True)

# Étape 2 : Nettoyage de l'espace au début de l'État_CodePostal
Dim_adresse['État_CodePostal'] = Dim_adresse['État_CodePostal'].str.strip()

# Étape 3 : Extraire uniquement le code postal
#Dim_adresse[['Code_postale', 'Etat']] = Dim_adresse['État_CodePostal'].astype(str).str.split(' ', expand=True)
Dim_adresse[['État', 'Code_postale']] = Dim_adresse['État_CodePostal'].astype(str).str.split(' ', expand=True)



In [22]:
Dim_adresse.drop(['État_CodePostal', 'Purchase_Address'], axis=1, inplace=True)


CHARGEMENT DES NOUVEAU DONNEES

In [23]:
Dim_date.to_csv("D_warehouse/Dim_date.CSV",index=False)
Dim_adresse.to_csv("D_warehouse/Dim_adresse.CSV",index=False)
Dim_commande.to_csv("D_warehouse/Dim_commande.CSV",index=False)
Ventes.to_csv("D_warehouse/Ventes.CSV",index=False)